In [1]:
import pandas as pd
import numpy as np

In [2]:
import meteostat
from meteostat import Hourly, Stations, Point
from datetime import datetime, timedelta

In [3]:
df = pd.read_csv("../raw_data/fire_incidents_data.csv")

In [4]:
td = timedelta(hours = 1)

In [5]:
location = pd.merge(df["TFS_Alarm_Time"],df["Latitude"], right_index = True,
               left_index = True)
display(location.head())
print(len(location))

,TFS_Alarm_Time,Latitude
0,2018-02-24T21:04:29,43.686558
1,2018-02-24T21:24:43,43.766135
2,2018-02-25T13:29:59,43.743230
3,2018-02-25T14:13:39,43.708659
4,2018-02-25T18:20:43,43.718118


25860


In [6]:
dfDim = pd.concat([location, df["Longitude"]], axis = 1)
display(dfDim.head())

,TFS_Alarm_Time,Latitude,Longitude
0,2018-02-24T21:04:29,43.686558,-79.599419
1,2018-02-24T21:24:43,43.766135,-79.390039
2,2018-02-25T13:29:59,43.743230,-79.245061
3,2018-02-25T14:13:39,43.708659,-79.478062
4,2018-02-25T18:20:43,43.718118,-79.443184


In [7]:
dfDim = dfDim.dropna()
display(dfDim.head())
print(len(dfDim))

,TFS_Alarm_Time,Latitude,Longitude
0,2018-02-24T21:04:29,43.686558,-79.599419
1,2018-02-24T21:24:43,43.766135,-79.390039
2,2018-02-25T13:29:59,43.743230,-79.245061
3,2018-02-25T14:13:39,43.708659,-79.478062
4,2018-02-25T18:20:43,43.718118,-79.443184


25858


In [8]:
timeKey = dfDim["TFS_Alarm_Time"]
timeKey = timeKey.apply(lambda x: str(x))
timeKey = timeKey.apply(lambda x:x.replace("-", ""))
timeKey = timeKey.apply(lambda x:x.replace(":", ""))
timeKey = timeKey.apply(lambda x:x.replace(" ", ""))
timeKey = timeKey.apply(lambda x:x.replace("T", ""))
display(timeKey.head())

0    20180224210429
1    20180224212443
2    20180225132959
3    20180225141339
4    20180225182043
Name: TFS_Alarm_Time, dtype: object

In [9]:
latKey = dfDim["Latitude"]
longKey = dfDim["Longitude"]
display(latKey.head())
print(len(latKey))

0    43.686558
1    43.766135
2    43.743230
3    43.708659
4    43.718118
Name: Latitude, dtype: float64

25858


In [10]:
latKey = latKey.apply(lambda x: x-43)
display(latKey.head())

0    0.686558
1    0.766135
2    0.743230
3    0.708659
4    0.718118
Name: Latitude, dtype: float64

In [11]:
latKey = latKey.apply(lambda x:x*10000)
display(latKey.head())

0    6865.58177
1    7661.34782
2    7432.30192
3    7086.59308
4    7181.18427
Name: Latitude, dtype: float64

In [12]:
latKey = latKey.apply(lambda x: int(x))
display(latKey.head())

0    6865
1    7661
2    7432
3    7086
4    7181
Name: Latitude, dtype: int64

In [13]:
display(longKey.head())

0   -79.599419
1   -79.390039
2   -79.245061
3   -79.478062
4   -79.443184
Name: Longitude, dtype: float64

In [14]:
longKey = longKey.apply(lambda x:x+79)
display(longKey.head())

0   -0.599419
1   -0.390039
2   -0.245061
3   -0.478062
4   -0.443184
Name: Longitude, dtype: float64

In [15]:
longKey = longKey.apply(lambda x:abs(x))
display(longKey.head())

0    0.599419
1    0.390039
2    0.245061
3    0.478062
4    0.443184
Name: Longitude, dtype: float64

In [16]:
longKey = longKey.apply(lambda x:x*10000)
longKey = longKey.apply(lambda x:int(x))
display(longKey.head())

0    5994
1    3900
2    2450
3    4780
4    4431
Name: Longitude, dtype: int64

In [17]:
longKey = longKey.apply(lambda x: str(x))
latKey = latKey.apply(lambda x: str(x))
print(type(latKey[0]))

<class 'str'>


In [18]:
weatherKey = pd.concat([timeKey, latKey, longKey], axis = 1)
display(weatherKey.head())

,TFS_Alarm_Time,Latitude,Longitude
0,20180224210429,6865,5994
1,20180224212443,7661,3900
2,20180225132959,7432,2450
3,20180225141339,7086,4780
4,20180225182043,7181,4431


In [19]:
weatherKey["Key"] = weatherKey.apply(lambda x:x["TFS_Alarm_Time"]+x["Latitude"]+x["Longitude"], axis = 1)
display(weatherKey.head())

,TFS_Alarm_Time,Latitude,Longitude,Key
0,20180224210429,6865,5994,2018022421042968655994
1,20180224212443,7661,3900,2018022421244376613900
2,20180225132959,7432,2450,2018022513295974322450
3,20180225141339,7086,4780,2018022514133970864780
4,20180225182043,7181,4431,2018022518204371814431


In [20]:
weatherKey = weatherKey[["Key"]]
display(weatherKey.head())

,Key
0,2018022421042968655994
1,2018022421244376613900
2,2018022513295974322450
3,2018022514133970864780
4,2018022518204371814431


In [21]:
weatherData = pd.concat([weatherKey, dfDim], axis = 1)
display(weatherData.head())

,Key,TFS_Alarm_Time,Latitude,Longitude
0,2018022421042968655994,2018-02-24T21:04:29,43.686558,-79.599419
1,2018022421244376613900,2018-02-24T21:24:43,43.766135,-79.390039
2,2018022513295974322450,2018-02-25T13:29:59,43.743230,-79.245061
3,2018022514133970864780,2018-02-25T14:13:39,43.708659,-79.478062
4,2018022518204371814431,2018-02-25T18:20:43,43.718118,-79.443184


In [22]:
print(type(weatherData["TFS_Alarm_Time"][0]))

<class 'str'>


In [23]:
weatherData["TFS_Alarm_Time"] = weatherData["TFS_Alarm_Time"].apply(lambda x:datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))
display(weatherData.head())
print(type(weatherData["TFS_Alarm_Time"][0]))

,Key,TFS_Alarm_Time,Latitude,Longitude
0,2018022421042968655994,2018-02-24 21:04:29,43.686558,-79.599419
1,2018022421244376613900,2018-02-24 21:24:43,43.766135,-79.390039
2,2018022513295974322450,2018-02-25 13:29:59,43.743230,-79.245061
3,2018022514133970864780,2018-02-25 14:13:39,43.708659,-79.478062
4,2018022518204371814431,2018-02-25 18:20:43,43.718118,-79.443184


<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [24]:
weatherData["StartTime"] = weatherData["TFS_Alarm_Time"].apply(lambda dt:dt.replace(second=0, microsecond=0, minute=0))
display(weatherData.head())

,Key,TFS_Alarm_Time,Latitude,Longitude,StartTime
0,2018022421042968655994,2018-02-24 21:04:29,43.686558,-79.599419,2018-02-24 21:00:00
1,2018022421244376613900,2018-02-24 21:24:43,43.766135,-79.390039,2018-02-24 21:00:00
2,2018022513295974322450,2018-02-25 13:29:59,43.743230,-79.245061,2018-02-25 13:00:00
3,2018022514133970864780,2018-02-25 14:13:39,43.708659,-79.478062,2018-02-25 14:00:00
4,2018022518204371814431,2018-02-25 18:20:43,43.718118,-79.443184,2018-02-25 18:00:00


In [26]:
weatherData["EndTime"] = weatherData["StartTime"].apply(lambda x:x+td)
display(weatherData.head())

,Key,TFS_Alarm_Time,Latitude,Longitude,StartTime,EndTime
0,2018022421042968655994,2018-02-24 21:04:29,43.686558,-79.599419,2018-02-24 21:00:00,2018-02-24 22:00:00
1,2018022421244376613900,2018-02-24 21:24:43,43.766135,-79.390039,2018-02-24 21:00:00,2018-02-24 22:00:00
2,2018022513295974322450,2018-02-25 13:29:59,43.743230,-79.245061,2018-02-25 13:00:00,2018-02-25 14:00:00
3,2018022514133970864780,2018-02-25 14:13:39,43.708659,-79.478062,2018-02-25 14:00:00,2018-02-25 15:00:00
4,2018022518204371814431,2018-02-25 18:20:43,43.718118,-79.443184,2018-02-25 18:00:00,2018-02-25 19:00:00


In [27]:
dfPoints = weatherData.apply(lambda x:meteostat.Point(x["Latitude"], x["Longitude"]), axis = 1)
display(dfPoints.head())

0    <meteostat.interface.point.Point object at 0x0...
1    <meteostat.interface.point.Point object at 0x0...
2    <meteostat.interface.point.Point object at 0x0...
3    <meteostat.interface.point.Point object at 0x0...
4    <meteostat.interface.point.Point object at 0x0...
dtype: object

In [28]:
dfHourlyData = pd.concat([dfPoints, weatherData["StartTime"], weatherData["EndTime"]], axis = 1)
dfHourlyData = dfHourlyData.rename(columns = {0:"Points"})
display(dfHourlyData.head())

,Points,StartTime,EndTime
0,<meteostat.interface.point.Point object at 0x0...,2018-02-24 21:00:00,2018-02-24 22:00:00
1,<meteostat.interface.point.Point object at 0x0...,2018-02-24 21:00:00,2018-02-24 22:00:00
2,<meteostat.interface.point.Point object at 0x0...,2018-02-25 13:00:00,2018-02-25 14:00:00
3,<meteostat.interface.point.Point object at 0x0...,2018-02-25 14:00:00,2018-02-25 15:00:00
4,<meteostat.interface.point.Point object at 0x0...,2018-02-25 18:00:00,2018-02-25 19:00:00


In [ ]:
weatherData["Weather"]= dfHourlyData.apply(lambda x:meteostat.Hourly(x['Points'], x['StartTime'],x['StartTime']), axis = 1)
display(weatherData.head())

In [ ]:
maxIndex = len(weatherData["Weather"])
print(maxIndex)
hourlyData = []
z = 0
for x in range(maxIndex):
    hourlyData.append(weatherData["Weather"][x].fetch())
    z = z+1
print(z)


In [ ]:
display(hourlyData[20])

In [41]:
dfHourly = dfHourly.reset_index()
display(dfHourly.head())

,level_0,index,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,0,1536080400000000000,25.8,17.2,59.0,NaN,NaN,70.0,11.2,NaN,1025.6,NaN,NaN
1,1,1627668000000000000,19.6,8.0,47.0,0.0,NaN,328.0,21.6,NaN,1016.4,NaN,2.0
2,2,1492556400000000000,5.0,-1.0,65.0,NaN,NaN,60.0,29.5,NaN,1027.2,NaN,NaN
3,3,1471492800000000000,22.2,19.0,82.0,NaN,NaN,80.0,9.4,NaN,1015.4,NaN,5.0
4,4,1405522800000000000,19.0,12.0,64.0,NaN,NaN,260.0,9.4,NaN,1011.3,NaN,NaN


In [42]:
dfHourly = dfHourly.drop(columns = {"level_0","index","tsun", "coco", "pres"})

In [45]:
display(dfHourly)
print(len(dfHourly))

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt
0,25.8,17.2,59.0,NaN,NaN,70.0,11.2,NaN
1,19.6,8.0,47.0,0.0,NaN,328.0,21.6,NaN
2,5.0,-1.0,65.0,NaN,NaN,60.0,29.5,NaN
3,22.2,19.0,82.0,NaN,NaN,80.0,9.4,NaN
4,19.0,12.0,64.0,NaN,NaN,260.0,9.4,NaN
5,17.9,15.7,87.0,0.0,NaN,166.0,12.6,NaN
6,6.2,-0.3,63.0,0.0,NaN,260.0,16.6,NaN


7


In [ ]:
#remove hourly stats

In [39]:
weatherData.to_csv("Weather.csv")

In [97]:
# problems: requesting data dropped some data from api
# the time range ie) 10 mins before + 10 mins after or?
# need to add comments
# cannot identify which fires are which anymore because of the change in times
#drop coco, tsun, pres